# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_file = os.path.join('..', 'WeatherPy', 'Resources', 'Cities.csv')

weather_df = pd.read_csv(weather_file)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aksu,41.1231,80.2644,85.64,20,0,9.04,CN,1658293787
1,Phan Thiet,10.9333,108.1000,83.66,73,100,9.35,VN,1658293787
2,Alberton,46.8168,-64.0654,65.05,87,36,20.58,CA,1658293787
3,Vaini,-21.2000,-175.2000,73.56,100,20,11.50,TO,1658293787
4,Nome,64.5011,-165.4064,46.47,71,100,5.75,US,1658293536
...,...,...,...,...,...,...,...,...,...
548,Pirganj,25.8523,88.3618,88.48,69,99,8.34,BD,1658293960
549,Kavieng,-2.5744,150.7967,83.34,71,100,5.23,PG,1658293960
550,Tambopata,-12.7333,-69.1833,75.33,77,100,2.01,PE,1658293960
551,Fenelon Falls,44.5334,-78.7496,69.57,80,90,5.19,CA,1658293961


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#Make locations and weights to configure
locations = weather_df[['Lat', 'Lng']]

#Change humidity from string to float to avoid ValueError
weights = weather_df['Humidity'].astype(float)
max_humidity = max(weather_df["Humidity"])

max_humidity

100

In [11]:
#adding heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity = max_humidity, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
new_weather_df = 

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
